In [307]:
import os, sys, time, datetime
import pandas as pd
import numpy as np

In [308]:
#importation des fichers de données
internet_data = pd.read_csv('data/internet_access_2017.csv')
density_data = pd.read_csv('data/population_density_2017.csv')
population_data = pd.read_csv('data/population_total_2017.csv')
superficie_data = pd.read_csv('data/superficie_2017.csv')

#importation du ficher metadata
metadata = pd.read_csv('data/metadata.csv')

In [309]:
internet_data.head(3)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
0,Aruba,ABW,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,62.0,69.0,74.000000,78.9,83.78,88.661227,93.542454,97.170000,NaN,NaN
1,Afghanistan,AFG,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,5.0,5.454545,5.9,7.00,8.260000,10.595726,11.447688,NaN,NaN
2,Angola,AGO,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,2.8,3.1,6.500000,8.9,10.20,12.400000,13.000000,14.339079,NaN,NaN


In [310]:
metadata

,Country Code,Region,IncomeGroup,SpecialNotes,TableName,Unnamed: 5
0,ABW,Latin America & Caribbean,High income,Central Bureau of Statistics and Central Bank ...,Aruba,NaN
1,AFG,South Asia,Low income,Central Statistics Organization; World Bank st...,Afghanistan,NaN
2,AGO,Sub-Saharan Africa,Lower middle income,IMF ; Source of population estimates: UN Popul...,Angola,NaN
3,ALB,Europe & Central Asia,Upper middle income,Albanian Institute of Statistics ; Source of p...,Albania,NaN
4,AND,Europe & Central Asia,High income,"Government of Andorra, Department of Statistic...",Andorra,NaN
5,ARB,NaN,NaN,Arab World aggregate. Arab World is composed o...,Arab World,NaN
6,ARE,Middle East & North Africa,High income,National Bureau of Statistics & Federal Compet...,United Arab Emirates,NaN
7,ARG,Latin America & Caribbean,High income,National Institute of Statistics and Census (I...,Argentina,NaN
8,ARM,Europe & Central Asia,Upper middle income,National Statistical Service of the Republic o...,Armenia,NaN
9,ASM,East Asia & Pacific,Upper middle income,BEA: https://www.bea.gov/national/gdp_territor...,American Samoa,NaN


In [311]:
#suppression des colonnes inutiles sur internet_data
internet = internet_data.loc[ :,["Country Name", "Country Code", "2017" ]].rename(index=str, columns={"2017": "internet"})
internet['internet'] = internet["internet"].round(1)


#on retire tout ce dont on n'a pas besoin sur le dataframe density_data
density = density_data.loc[ :,["Country Name","2017" ]].round(2).rename(index=str, columns={"2017": "density"})

#on retire tout ce dont on n'a pas besoin sur le dataframe population_data
population = population_data.loc[ :,["Country Name","2017" ]].rename(index=str, columns={"2017": "population"})
population['population'] = population["population"].div(1000000).round(2)

#On retire tout ce dont on n'a pas besoin sur le dataframe superficie_data
superficie_latest = superficie_data.loc[ :,["Country Name", "2017" ]].rename(index=str, columns={"2017": "superficie"})
superficie_latest['superficie'] = superficie_latest["superficie"].div(1000).round(3)


#suppression des colonnes inutiles sur metadata
metadata = metadata.iloc[:,:-1]
metadata = metadata.drop('SpecialNotes', axis=1)
metadata = metadata.rename(index=str, columns={"TableName": "Country Name"})

#remplassement des valeurs NAN par Other dans les Categories Region et IncomeGroup
values = {'Region': 'Other', 'IncomeGroup': 'Other'}
metadata.loc[:,['Region', 'IncomeGroup']] = metadata.loc[:,['Region', 'IncomeGroup']].fillna(value=values)

In [312]:
merged = pd.merge(internet, density, how='left', on=['Country Name'])
merged = pd.merge(merged, population, how='left', on=['Country Name'])
merged = pd.merge(merged, superficie_latest, how='left', on=['Country Name'])

#puis on fusionne avec le dataset metadate
merged = pd.merge(merged, metadata, how='left', on=['Country Name'])

In [313]:
#on dégage toutes les lignes invalides (internet, superficie, population = 0)
merged = merged.loc[merged['internet'] > 0,:].loc[merged['superficie'] > 0,:].loc[merged['population'] > 0,:]

In [314]:
#on met en forme notre tableaux
pivot_internet = pd.pivot_table(merged, values=['population', 'superficie', 'density', 'internet'],index=['Region', 'Country Name', 'IncomeGroup'],
                            fill_value=0)


In [315]:
#on créer des tableaux pour visualiser nos données
stats_internet = pivot_internet[['population', 'superficie', 'density', 'internet']].sort_values(['Region','internet'], ascending=[1,0])
stats_superficie = pivot_internet[['population', 'density', 'superficie', 'internet']].sort_values(['Region','superficie'], ascending=[1,0])
stats_density = pivot_internet[['population', 'superficie', 'density', 'internet']].sort_values(['Region','density'], ascending=[1,0])
stats_population = pivot_internet[['superficie', 'density', 'population', 'internet']].sort_values(['Region','population'], ascending=[1,1])


In [318]:
#Example: On récupère les 10 pays d'Afrique Sub-Saharienne classés par accès à internet en 2017
stats_internet.loc['Sub-Saharan Africa'].head(10)

,,population,superficie,density,internet
Country Name,IncomeGroup,,,,
Seychelles,High income,0.10,0.46,208.35,58.8
Cabo Verde,Lower middle income,0.55,4.03,135.58,57.2
South Africa,Upper middle income,56.72,1213.09,46.75,56.2
Mauritius,Upper middle income,1.26,2.03,622.96,55.6
Gabon,Upper middle income,2.03,257.67,7.86,50.3
Botswana,Upper middle income,2.29,566.73,4.04,41.4
Ghana,Lower middle income,28.83,227.54,126.72,37.9
Namibia,Upper middle income,2.53,823.29,3.08,36.8
Eswatini,Lower middle income,1.37,17.20,79.49,30.3


In [317]:
#on peut aussi ranger tous nos tableaux dans un dict
stats = {}
stats['internet'] = pivot_internet[['population', 'superficie', 'density', 'internet']].sort_values(['Region','internet'], ascending=[1,0])
stats['superficie'] = pivot_internet[['population', 'density', 'superficie', 'internet']].sort_values(['Region','superficie'], ascending=[1,0])
stats['density'] = pivot_internet[['population', 'superficie', 'density', 'internet']].sort_values(['Region','density'], ascending=[1,0])
stats['population'] = pivot_internet[['superficie', 'density', 'population', 'internet']].sort_values(['Region','population'], ascending=[1,1])


In [298]:
#Example: On récupère les 10 pays Européens classés par densité de population en 2017
stats['density'].loc['Europe & Central Asia'].head(10)

,,population,superficie,density,internet
Country Name,IncomeGroup,,,,
Monaco,High income,0.04,0.002,19347.50,97.1
San Marino,High income,0.03,0.060,556.67,60.2
Netherlands,High income,17.13,33.690,508.50,93.2
Belgium,High income,11.38,30.280,375.90,87.7
United Kingdom,High income,66.02,241.930,272.90,94.6
Luxembourg,High income,0.60,2.430,245.41,97.8
Liechtenstein,High income,0.04,0.160,237.01,98.1
Germany,High income,82.69,349.360,236.68,84.4
Switzerland,High income,8.45,39.516,213.86,93.7


In [299]:
#on peut alors exporter nos données vers un fichier .csv
merged.to_csv('data/merged.csv')

In [300]:
#on peut aussi exporter nos données dans un fichier JSON:
import json
stats_json = merged.to_json(orient='table')
with open('data/stats_2017.json', 'w') as fp:
    json.dump(stats_json, fp)

In [345]:
#La liste des 10 pays connectés au monde
stats_internet.sort_values('internet', ascending=0).head(10)

population  superficie  \
Region                     Country Name  IncomeGroup                           
Europe & Central Asia      Andorra       High income        0.08       0.470   
North America              Bermuda       High income        0.07       0.054   
Europe & Central Asia      Iceland       High income        0.34     100.250   
                           Liechtenstein High income        0.04       0.160   
Middle East & North Africa Kuwait        High income        4.14      17.820   
Europe & Central Asia      Luxembourg    High income        0.60       2.430   
                           Faroe Islands High income        0.05       1.396   
Latin America & Caribbean  Aruba         High income        0.11       0.180   
Europe & Central Asia      Monaco        High income        0.04       0.002   
                           Denmark       High income        5.76      41.990   

                                                       density  internet  
Region                     Country Name  IncomeGroup                      
Europe & Central Asia      Andorra       High income    163.76      98.9  
North America              Bermuda       High income   1211.87      98.4  
Europe & Central Asia      Iceland       High income      3.43      98.3  
                           Liechtenstein High income    237.01      98.1  
Middle East & North Africa Kuwait        High income    232.13      98.0  
Europe & Central Asia      Luxembourg    High income    245.41      97.8  
                           Faroe Islands High income     35.31      97.6  
Latin America & Caribbean  Aruba         High income    584.80      97.2  
Europe & Central Asia      Monaco        High income  19347.50      97.1  
                           Denmark       High income    137.29      97.1

In [320]:
#on peut aussi appliquer des masks
internet_masked = internet_data.loc[ :,["Country Name", "Country Code","1990", "2000",  "2010", "2017" ]]
internet_masked = internet_masked.round(1)

merged_masked = pd.merge(internet_masked, metadata, how='left', on=['Country Code'])

pivot_masked = pd.pivot_table(internet_masked, values=["1990", "2000",  "2010", "2017"],index= 'Country Name',
                            fill_value=0)
masked = pivot_masked.mask(pivot_masked < 100, "full").mask(pivot_masked < 90, "large" ).mask(pivot_masked < 50, "low").mask(pivot_masked < 10, "none" )

masked_all = pd.merge(masked, superficie_latest, how='outer', on=['Country Name'], left_index = True)
masked_all = pd.merge(masked_all, population, how='outer', on=['Country Name'], left_index = True )
masked_all = pd.merge(masked_all, density, how='outer', on=['Country Name'], left_index = True )

masked_all = masked_all.dropna()

In [344]:
masked_all.sort_values(['2017','superficie', 'population', 'density'], ascending=[1,1,1,1]).head(10)

,Country Name,1990,2000,2010,2017,superficie,population,density
147,Monaco,none,low,large,full,0.002,0.04,19347.50
25,Bermuda,none,low,large,full,0.054,0.07,1211.87
135,Liechtenstein,none,low,large,full,0.160,0.04,237.01
0,Aruba,none,low,large,full,0.180,0.11,584.80
4,Andorra,none,low,large,full,0.470,0.08,163.76
20,Bahrain,none,none,large,full,0.778,1.49,1918.49
76,Faroe Islands,none,low,large,full,1.396,0.05,35.31
142,Luxembourg,none,low,full,full,2.430,0.60,245.41
29,Brunei Darussalam,none,none,large,full,5.270,0.43,81.35
198,Qatar,none,none,large,full,11.610,2.64,227.32
